In [1]:
# std lib
from getpass import getpass

# 3rd party
import numpy as np
import pylab as plt
%matplotlib inline
import pandas as pd

# Data Lab
from dl import authClient as ac, queryClient as qc #, storeClient as sc
from dl.helpers.utils import convert

from astropy.table import Table
from tqdm import tqdm

In [6]:
print(qc.schema('ls_dr10.tractor'))


Schema: ls_dr10
 Table: tractor

     Column Name   Description
     -----------   -----------
         release   Integer denoting the camera and filter set used, which wil
                   l be unique for a given processing run of the data (as doc
                   umented here_)
         brickid   Brick ID [1,662174]
       brickname   Name of brick, encoding the brick sky position, eg 1126p22
                   2 near RA=112.6, Dec=+22.2
           objid   Catalog object number within this brick; a unique identifi
                   er hash is release,brickid,objid;  objid spans [0,N-1] and
                   is contiguously enumerated within each brick
   brick_primary   True if the object is within the brick boundary
        maskbits   Bitwise mask indicating that an object touches a pixel in 
                   the coadd/*/*/*maskbits* maps, as cataloged on the DR10 bi
                   tmasks page_
         fitbits   Bitwise mask detailing pecularities of how an object was 

In [5]:
def tractor(ra0, dec0, name):    
    query = f'''SELECT brickid, brickname, objid, type, ra, dec, ebv, flux_g, flux_r, flux_i, flux_z, psfsize_g, psfsize_r, psfsize_i, psfsize_z, sersic, sersic_ivar, shape_r, shape_r_ivar, shape_e1, shape_e1_ivar, shape_e2, shape_e2_ivar, mag_g, mag_r, mag_i, mag_z, mag_w1, mag_w2, mag_w3, mag_w4, g_r, r_i, r_z, i_z FROM ls_dr10.tractor WHERE q3c_radial_query(ra, dec, {ra0}, {dec0}, 0.133)'''
    response = qc.query(sql=query, format='csv')
    df = convert(response,'pandas')
    df.to_csv(f'/home/seba/Documents/DECALS/belen_joined/ngc1313.csv', index=False)

In [4]:
tractor(49.566875, -66.498250)

In [ ]:
query = '''SELECT brickid, brickname, objid, type, ra, dec, ebv, flux_g, flux_r, flux_i, flux_z, psfsize_g, psfsize_r, psfsize_i, psfsize_z, sersic, sersic_ivar, shape_r, shape_r_ivar, shape_e1, shape_e1_ivar, shape_e2, shape_e2_ivar, mag_g, mag_r, mag_i, mag_z, mag_w1, mag_w2, mag_w3, mag_w4, g_r, r_i, r_z, i_z FROM ls_dr10.tractor'''


In [8]:
grupos_decals = Table.read('/home/seba/Documents/DECALS/groups_in_decals.csv')
RA = grupos_decals['ra']
DEC = grupos_decals['dec']
NAME = grupos_decals['Group']

In [5]:
control_sample = Table.read('/home/seba/Documents/DECALS/galaxias_extra.csv')
RA_CS= control_sample['ra']
DEC_CS = control_sample['dec']
NAME_CS = control_sample['index']

In [6]:
for ra0, dec0, name in tqdm(zip(RA_CS, DEC_CS, NAME_CS), total=len(RA_CS)):
    tractor(ra0, dec0, name)

100%|█████████████████████████████████████████████████████████████████████████████| 60/60 [00:59<00:00,  1.01it/s]


In [25]:
ser = Table.read('/home/seba/Documents/CG/Hernandez-Fernandez/Galaxy_catalogue.csv')
groups=[20, 26, 235]
filtered_table = ser[(ser['Group'] == 20) | (ser['Group'] == 26) | (ser['Group'] == 235)]
columns=filtered_table['_RAJ2000', '_DEJ2000', 'Group', 'Gal']
columns.write('tres_grupos.csv', format='csv')